<a href="https://colab.research.google.com/github/robinkelsick/kelsickdata/blob/main/Quantium_Chip_Category_Customer_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author:** Robin Kelsick II

**Portfolio:** https://kelsickdata.space  

**Tools:** Python, pandas, matplotlib, Google Colab


# Quantium Retail Analytics – Chip Category

## Objective
Understand customer segments and purchasing behaviour for the chip category to inform strategic planning for the next half year.

## Data Sources
- Transaction data
- Customer purchase behaviour data


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for pandas DataFrames to show all columns
pd.set_option('display.max_columns', None)

# Load transaction data from an Excel file
transactions = pd.read_excel("/QVI_transaction_data.xlsx")
# Load customer purchase behavior data from a CSV file
customers = pd.read_csv("/QVI_purchase_behaviour.csv")

# Display the first few rows of the transactions DataFrame
print("Transactions Head:")
print(transactions.head())

# Display the first few rows of the customers DataFrame
print("Customers Head:")
print(customers.head())

## Data Quality Check – Transaction Data

Initial review of the transaction dataset to identify missing values, inconsistencies, and outliers.


In [ ]:
# Display concise summary of the DataFrame, including data types and non-null values
transactions.info()
# Count the number of missing values in each column
transactions.isnull().sum()
# Generate descriptive statistics of the DataFrame
transactions.describe()

### Outlier Identification
An extreme outlier was identified in product quantity, with a maximum value of 200 units compared to an average of approximately 2 units per transaction.  
This discrepancy indicates non-typical purchasing behavior and warranted exclusion from the analysis.


In [ ]:
# Get descriptive statistics for 'TOT_SALES' and 'PROD_QTY' columns
transactions[['TOT_SALES', 'PROD_QTY']].describe()

# Display transactions with the highest 'PROD_QTY'
transactions.sort_values('PROD_QTY', ascending=False).head()
# Display transactions with the highest 'TOT_SALES'
transactions.sort_values('TOT_SALES', ascending=False).head()

# Identify and display transactions where 'PROD_QTY' is greater than 50 (potential outliers)
transactions[transactions['PROD_QTY'] > 50]
# Filter out transactions where 'PROD_QTY' is greater than 50 to create a clean dataset
transactions_clean = transactions[transactions['PROD_QTY'] <= 50]
# Get descriptive statistics for 'PROD_QTY' and 'TOT_SALES' in the cleaned DataFrame
transactions_clean[['PROD_QTY', 'TOT_SALES']].describe()

### Convert `DATE` column to datetime

The `DATE` column is currently stored as an integer, likely in Excel serial date format. We need to convert it to a datetime object for proper date-based analysis.

In [ ]:
# Verify the conversion of the 'DATE' column to datetime objects
print(transactions_clean['DATE'].head())
# Display concise summary of the cleaned DataFrame to confirm data types
transactions_clean.info()

### Outlier Handling
Two transactions were identified with unusually high product quantities (200 units per transaction), likely indicating non-household or commercial purchasing behavior.  
These transactions were removed from the dataset to ensure the analysis reflects typical customer purchasing patterns.


## Data Quality Check – Customer Data

Review of customer purchase behaviour data to identify missing values, inconsistencies, or formatting issues prior to merging with transaction data.


In [ ]:
# Display concise summary of the customer DataFrame
customers.info()
# Count missing values in each column of the customer DataFrame
customers.isnull().sum()
# Display the first few rows of the customer DataFrame
customers.head()

In [ ]:
# Check the number of unique loyalty card numbers and compare to total rows to find duplicates
customers['LYLTY_CARD_NBR'].nunique(), customers.shape[0]

In [ ]:
# Count occurrences of each unique value in the 'LIFESTAGE' column
customers['LIFESTAGE'].value_counts()
# Count occurrences of each unique value in the 'PREMIUM_CUSTOMER' column
customers['PREMIUM_CUSTOMER'].value_counts()

### Customer Data Summary
The customer dataset contains one record per loyalty card with no missing values in key segmentation fields.  
Lifestage and premium customer segments are clearly defined and suitable for merging with transaction data.


## Data Preparation – Merging Datasets

The cleaned transaction data was merged with customer purchase behaviour data using the loyalty card number to enable customer-level analysis.


In [ ]:
# Merge the cleaned transactions data with customer data using 'LYLTY_CARD_NBR' as the key
merged_data = transactions_clean.merge(
    customers,
    on='LYLTY_CARD_NBR',
    how='inner' # Use inner join to keep only matching records
)

# Display concise summary of the merged DataFrame
merged_data.info()
# Display the first few rows of the merged DataFrame
merged_data.head()

In [ ]:
# Check for any missing values in the merged DataFrame
merged_data.isnull().sum()

In [ ]:
# Export the merged DataFrame to a CSV file without the index
merged_data.to_csv("merged_data.csv", index=False)

### Merged Dataset Summary
Transaction and customer datasets were successfully merged using loyalty card numbers.  
The resulting dataset is complete and ready for customer segmentation and sales analysis.


## Core Metrics Overview

This section defines the key metrics used to assess customer purchasing behaviour within the chip category.


In [ ]:
# Calculate total sales from the merged data
total_sales = merged_data['TOT_SALES'].sum()

# Calculate the total number of transactions (rows) in the merged data
total_transactions = merged_data.shape[0]

# Calculate the average sales per transaction
avg_sales_per_txn = merged_data['TOT_SALES'].mean()

# Calculate the average product quantity per transaction
avg_qty_per_txn = merged_data['PROD_QTY'].mean()

# Display the calculated core metrics
total_sales, total_transactions, avg_sales_per_txn, avg_qty_per_txn

## Sales Analysis by Customer Lifestage


In [ ]:
# Group merged data by 'LIFESTAGE' and calculate aggregate sales metrics
sales_by_lifestage = (
    merged_data
    .groupby('LIFESTAGE')
    .agg(
        total_sales=('TOT_SALES', 'sum'), # Total sales for each lifestage
        avg_sales_per_txn=('TOT_SALES', 'mean'), # Average sales per transaction for each lifestage
        avg_qty=('PROD_QTY', 'mean'), # Average product quantity per transaction for each lifestage
        transactions=('TOT_SALES', 'count') # Number of transactions for each lifestage
    )
    .sort_values('total_sales', ascending=False) # Sort results by total sales in descending order
)

# Display the sales analysis by lifestage
sales_by_lifestage

## Sales Analysis by Premium Customer Segment

In [ ]:
# Group merged data by 'PREMIUM_CUSTOMER' and calculate aggregate sales metrics
sales_by_premium = (
    merged_data
    .groupby('PREMIUM_CUSTOMER')
    .agg(
        total_sales=('TOT_SALES', 'sum'), # Total sales for each premium customer segment
        avg_sales_per_txn=('TOT_SALES', 'mean'), # Average sales per transaction for each premium customer segment
        avg_qty=('PROD_QTY', 'mean'), # Average product quantity per transaction for each premium customer segment
        transactions=('TOT_SALES', 'count') # Number of transactions for each premium customer segment
    )
    .sort_values('total_sales', ascending=False) # Sort results by total sales in descending order
)

# Display the sales analysis by premium customer segment
sales_by_premium

In [ ]:
# Create a bar plot to visualize total sales by customer lifestage
plt.figure(figsize=(10,6))
sns.barplot(
    x=sales_by_lifestage.index,
    y=sales_by_lifestage['total_sales']
)
plt.xticks(rotation=45) # Rotate x-axis labels for better readability
plt.title("Total Sales by Customer Lifestage") # Set plot title
plt.ylabel("Total Sales") # Set y-axis label
plt.xlabel("Lifestage") # Set x-axis label
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
# Create a bar plot to visualize total sales by premium customer segment
plt.figure(figsize=(6,4))
sns.barplot(
    x=sales_by_premium.index,
    y=sales_by_premium['total_sales']
)
plt.title("Total Sales by Premium Customer Segment") # Set plot title
plt.ylabel("Total Sales") # Set y-axis label
plt.xlabel("Customer Type") # Set x-axis label
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
# Export the sales by lifestage DataFrame to a CSV file
sales_by_lifestage.to_csv("sales_by_lifestage.csv")
# Export the sales by premium customer DataFrame to a CSV file
sales_by_premium.to_csv("sales_by_premium.csv")

## Pack Size Analysis by Customer Segment

This section explores whether purchasing behaviour differs by pack size across customer segments.


In [ ]:
# Extract pack size (numeric value) from the 'PROD_NAME' column using a regular expression
# Convert the extracted pack size to float data type
merged_data['PACK_SIZE'] = merged_data['PROD_NAME'].str.extract(r'(\d+)').astype(float)

# Display the product name and extracted pack size for verification
merged_data[['PROD_NAME', 'PACK_SIZE']].head()

In [ ]:
# Generate descriptive statistics for the 'PACK_SIZE' column
merged_data['PACK_SIZE'].describe()

In [ ]:
# Group merged data by 'LIFESTAGE' and calculate the mean 'PACK_SIZE'
# Sort the results in descending order of mean pack size
pack_by_lifestage = (
    merged_data
    .groupby('LIFESTAGE')['PACK_SIZE']
    .mean()
    .sort_values(ascending=False)
)

# Display the average pack size by lifestage
pack_by_lifestage

In [ ]:
# Group merged data by 'PREMIUM_CUSTOMER' and calculate the mean 'PACK_SIZE'
# Sort the results in descending order of mean pack size
pack_by_premium = (
    merged_data
    .groupby('PREMIUM_CUSTOMER')['PACK_SIZE']
    .mean()
    .sort_values(ascending=False)
)

# Display the average pack size by premium customer segment
pack_by_premium

### Key Insights
- Older Singles/Couples contribute the highest total sales, driven by frequent purchasing behaviour.
- Mainstream customers exhibit the highest average spend per transaction, indicating a preference for larger pack sizes.
- Pack size analysis confirms that higher spending among mainstream customers is primarily driven by larger volume purchases rather than higher-priced products.


## Exported Files

Here are the CSV files and visualizations that have been exported:

- `merged_data.csv`
- `sales_by_lifestage.csv`
- `sales_by_premium.csv`
- `total_sales_by_lifestage.png`
- `total_sales_by_premium_customer.png`

## Conclusion

This analysis successfully identified key customer segments and their purchasing behaviors within the chip category. Older Singles/Couples and Mainstream customers are significant contributors to total sales and average transaction values, respectively. The data shows no substantial difference in pack size preferences across lifestages or premium customer segments, suggesting that the higher sales for Mainstream customers are driven by higher purchase volume rather than larger pack sizes per se. These insights provide a strong foundation for targeted marketing and promotional strategies.

In [ ]:
# Create a bar plot to visualize total sales by customer lifestage
plt.figure(figsize=(10,6))
sns.barplot(
    x=sales_by_lifestage.index,
    y=sales_by_lifestage['total_sales']
)
plt.xticks(rotation=45) # Rotate x-axis labels for better readability
plt.title("Total Sales by Customer Lifestage") # Set plot title
plt.ylabel("Total Sales") # Set y-axis label
plt.xlabel("Lifestage") # Set x-axis label
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.savefig("total_sales_by_lifestage.png") # Save the plot as a PNG file
plt.show()

In [ ]:
# Create a bar plot to visualize total sales by premium customer segment
plt.figure(figsize=(6,4))
sns.barplot(
    x=sales_by_premium.index,
    y=sales_by_premium['total_sales']
)
plt.title("Total Sales by Premium Customer Segment") # Set plot title
plt.ylabel("Total Sales") # Set y-axis label
plt.xlabel("Customer Type") # Set x-axis label
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.savefig("total_sales_by_premium_customer.png") # Save the plot as a PNG file
plt.show()

## Exported Files

Here are the CSV files and visualizations that have been exported:

- `merged_data.csv`
- `sales_by_lifestage.csv`
- `sales_by_premium.csv`
- `total_sales_by_lifestage.png`
- `total_sales_by_premium_customer.png`

## Recommendations

- Focus promotional and marketing efforts on Mainstream Older Singles/Couples, as they represent the highest total sales and strongest per-transaction spend.
- Prioritise availability and promotion of medium-to-large chip pack sizes to align with mainstream purchasing preferences.
- Use targeted promotions (e.g. multi-buy offers) to encourage continued volume purchases within this segment.
